![1](ATLASOD.gif)

## Antra pamoka

Šioje pamokoje mes:
* Tęsime ATLAS atvirų duomenų analizę su tikrais ir simuliuotais duomenimis
* Atliksime duomenų glotninimą pratimą, kad išmatuoti Z bozono masę

In [3]:
import ROOT
%jsroot on
ROOT.gStyle.SetOptStat(0) # panaikina histogramos statistikos dėžę

Welcome to JupyROOT 6.26/06


In [4]:
# Šį kartą analizuosime tikrus duomenis!
duom_failas = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_A.2lep.root")
sim_failas = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363356.ZqqZll.2lep.root")

In [5]:
# Bandysime palyginti tikrus su simuliuotais duomenimis.
# Kad veltui nekartoti to paties skaičiavimo, apibrėžkime funkciją
def gauti_masės_histogramą(failas, pavadinimas, n_įvykių, intervalai=100, min_masė=5, max_masė=130):
    print(f"Skaitome {n_įvykių} įvykių iš failo {failas.GetName()}")
    medis = failas.Get("mini")
    
    masės_histograma = ROOT.TH1D(pavadinimas, 
                                 "; leptonų sistemos masė [GeV]; įvykių skaičius",
                                 intervalai, min_masė, max_masė)

    # Pastorinkime histogramą kad aiškiau matytųsi grafike
    masės_histograma.SetLineWidth(2)

    pirmas_leptonas = ROOT.TLorentzVector()
    antras_leptonas = ROOT.TLorentzVector()

    n = 0
    # Panašiai, kaip ir pirmoje pamokoje
    for įvykis in medis:
        if įvykis.lep_n < 2:
            continue

        # Leptonų krūviai privalo būti skirtingi
        if (abs(įvykis.lep_type[0]) == 13 and abs(įvykis.lep_type[1]) == 13):

            pirmo_pt = įvykis.lep_pt[0]/1000.
            antro_pt = įvykis.lep_pt[1]/1000.
            pirmo_E = įvykis.lep_E[0]/1000.
            antro_E = įvykis.lep_E[1]/1000.
            pirmo_phi = įvykis.lep_phi[0]
            antro_phi = įvykis.lep_phi[1]
            pirmo_eta = įvykis.lep_eta[0]
            antro_eta = įvykis.lep_eta[1]

            pirmas_leptonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
            antras_leptonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

            sistema = pirmas_leptonas + antras_leptonas
            masė = sistema.M()

            masės_histograma.Fill(masė)
            n += 1
            if n == n_įvykių:
                break

    # Grąžiname histogramą naudotojui
    return masės_histograma

In [6]:
# Norime dviejų histogramų, vienos duomenims ir kitos simuliacijai.
# Simuliacija naudojama jog įsitikintume, kad duomenyse matome tinkamą procesą ir kad detektorius veikia kaip mes tikimės
# Panaudokime savo sukurtą funkciją du kartus
duom_histograma = gauti_masės_histogramą(duom_failas, "duomenys", 100000)
sim_histograma = gauti_masės_histogramą(sim_failas, "simuliacija", 100000)

Skaitome 100000 iš failo https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_A.2lep.root
Skaitome 100000 iš failo https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363356.ZqqZll.2lep.root


In [ ]:
drobė = ROOT.TCanvas()
# Drobė savaime būna tuščia, taigi nupiešiame ir histogramą
drobė.Draw()
duom_histograma.SetLineColor(ROOT.kBlack)
duom_histograma.SetMarkerColor(ROOT.kBlack)
duom_histograma.Draw("PE") 
# PE = point + error = taškai su paklaidomis
sim_histograma.Draw("SAME")

In [ ]:
# Nupieškime šį patį grafiką su logaritmine y ašimi
drobė_log = ROOT.TCanvas()
drobė_log.SetLogy()
duom_histograma.Draw("PE") 
sim_histograma.Draw("SAME")
drobė_log.Draw("")

Matome, kad simuliacija gana gerai atitinka tikrus duomenis, taigi galime būti tikri kad matome Z bozoną!  
Vienas simuliacijos mėginys visada modeliuoja tam tikrą Feinmano diagramą, šiuo atveju $Z\to \mu\mu$  .


Ar pastebėjote antrą rezonansą 9 GeV aplinkoje. Kadangi tai nematoma simuliacijoje, tai tikrai yra kitas visiškai nesusijęs fizikinis procesas. Kokios dalelės rezonansas tai galėtų būti?  

In [ ]:
histograma = gauti_masės_histogramą(duom_failas, "duomenys", 10000, intervalai=50, min_masė=70, max_masė=110)

In [ ]:
glotninimo_drobė = ROOT.TCanvas()
glotninimo_drobė.Draw()
gausas = ROOT.TF1("gausas", "gaus")
# Pakeiskime parametrų pavadinimus:
gausas.SetParNames("Amplitudė","Vidurkis","Standartinis nuokrypis")

histograma.Fit("gausas")

# Sakykime, kad norime sukurti savo pačių funkciją aprašyti rezonansą. 
# Paprastas pavyzdys: pridėkime tiesinę funkciją ir konstantą prie Gauso skirstinio
funkcija = ROOT.TF1("funkcija", "gausas + [0] + [1]*x")
funkcija.SetParName(0, "Konstanta")
funkcija.SetParName(1, "Tiesės koeficientas")
funkcija.SetLineColor(ROOT.kOrange)

# Kad patikrinti funkciją
print("Naudojama glotninimo funkcija:", funkcija.GetFormula().GetExpFormula())


histograma.Fit("funkcija", "+") # + reikalingas kad neištrinti praeitos funkcijos 

centras = gausas.GetParameter(1)
gausas.SetLineColor(ROOT.kRed)

#histograma.Draw("PE")
#funkcija.Draw("SAME")
#gausas.Draw("SAME")

# Sukurkime grafiko legendą
legenda = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legenda.AddEntry(histograma, "Duomenys")
legenda.AddEntry(gausas, "Gausas")
legenda.AddEntry(funkcija, "Gausas + tiesė")
legenda.Draw("")

# Užrašykime gautą rezultatą ant drobės
tekstas_nma = ROOT.TText(0.15, 0.8, "NMA ATLAS Atvirų duomenų analizė")
tekstas_nma.SetTextSize(13)
tekstas_nma.SetNDC()
tekstas_nma.Draw()

tekstas_glautninimas = ROOT.TText(0.15, 0.6, f"rasta Z bozono masė: {centras:.2f} GeV")
tekstas_glautninimas.SetTextSize(13)
tekstas_glautninimas.SetNDC()
tekstas_glautninimas.Draw()

Duomenų glotninimas labai dažnai naudojamas dalelių fizikos statistinė analizėje aprašant dalelių rezonansus ir ne tik. Daugiau informacijos ROOT puslapyje apie glotninimus: https://root.cern/manual/fitting/ (žiūrėti python o ne C++ instrukcijas bei pavyzdžius)  
__Užduotys__:

* Paieškokite internete kokia išmatuota Z bozono masė. Kodėl mūsų gautas rezultatas galėtų skirtis?
* Realistiškiau dalelių rezonansus aprašo ne Gauso skirstinys, bet Breito-Wignerio formulė (paveikslas apačioje). Aprašykite šią funkciją, panašiai kaip padarėme su Gausu + tiese (oranžinis grafikas) ir atlikite glotninimą. Ar rezultatas atrodo geriau?
[<img src="Breit-Wigner.png" width="300"/>](Breit-Wigner.png)
* Apskaičiuokite glotninimo būdu ne tik rezonanso centrą, bet ir jo storį, $\Gamma$. Apskaičiuokite dalelės gyvavimo trukmę (pusamžį) pasinaudodami formule $\tau=\hbar/\Gamma$. Galite pasinaudoti $\hbar = 6.6 \times 10^{-16} \mathrm{eVs}$. Palyginkite savo gautą rezultatą su oficialiai išmatuotu. (Komentaras: Z bozonas gali skilti ir į kitas daleles, kad tiksliai apskaičiuoti pilną pusamžį iš tikrųjų reiktų išmatuoti visus skilimo kanalus, tačiau čia tuo neapsiimsime). 
* Pakartokite glautninimą pakeisdami histogramos intervalų skaičių (į pvz. 10, 1000) ir ribas. Ką pastebėjote?
* Pakartokite užduotį pasirinkdami elektronų poras vietoj miuonų. Ar glotninimo rezultatai palyginami?



Žemiau esančioje celėje sukuriame naują masės histogramą kur $M<15\,\mathrm{GeV}$.  

In [ ]:
# Įskaitome visus esančius duomenis (-1)
upsilon_histograma = gauti_masės_histogramą(duom_failas, "upsilon", -1, intervalai=100, min_masė=7, max_masė=12)
drobė_upsilon = ROOT.TCanvas()
drobė_upsilon.Draw("")
upsilon_histograma.Draw("PE") 

Iš tikrųjų, matome dvi skirtingas persiklojančias būsenas: $\Upsilon(1S)$ ir sužadintą $\Upsilon(2S)$, kas apsunkina padėtį.

__Papildoma užduotis__:  Pabandykite glotninimo būdu surasti $\Upsilon$ rezonansų masę.  



_Patarimas:_ Kad sukurti naują funkciją, sudarytą iš dviejų skirtingų normalių skirstinių, galima padaryti taip:
```
f1 = ROOT.TF1("f1","gaus")
f2 = ROOT.TF1("f2","gaus")
f = ROOT.TF1("suma", "f1+f2")
```